In [1]:
#import lxml.html
from lxml import etree
import codecs
import xml.etree.ElementTree as ET
import pandas as pd
import time

In [2]:
val_file = "/scratch/users/ocan13/hyperpartisan/data/xml/train/articles-training-20180831.xml"
tree = ET.parse(val_file)
root = tree.getroot()

In [3]:
def stringify_children(node):
    s = node.text
    if s is None:
        s = ''
    for child in node:
        s += ' ' + stringify_children(child)
    return s

In [ ]:
df = pd.DataFrame(columns=['id','publish_time','title','text'])

for el in root:
    df = df.append({'id':el.attrib['id'], 'publish_time':el.attrib['published-at'], 'title':el.attrib['title'], 'text':"".join([x for x in el.itertext()])},ignore_index=True)

In [3]:
val_df = pd.DataFrame([el for el in root], columns=['xml'])

val_df['id'] = None
val_df['publish_time'] = None
val_df['title'] = None
val_df['text'] = None

def get_stuff(row):

    row.id = row.xml.attrib['id']
    try:
        row.publish_time = row.xml.attrib['published-at']
    except:
        row.publish_time = ''

    row.title = row.xml.attrib['title']
    row.text = "".join([x for x in row.xml.itertext()])
    try:
        row.xml = etree.tostring(row.xml)
    except:
        row.xml = ''
    return row

val_df = val_df.apply(get_stuff, axis=1)
val_df.drop(columns=['xml'], inplace=True)

In [4]:
len(val_df[val_df.publish_time != ''])

686082

In [5]:
val_file_labels = "/scratch/users/ocan13/hyperpartisan/data/ground-truth/train/ground-truth-training-20180831.xml"
tree = ET.parse(val_file_labels)
root = tree.getroot()

In [6]:
val_df2 = pd.DataFrame([el for el in root], columns=['xml'])

val_df2['id'] = None
val_df2['hyperpartisan'] = None
val_df2['bias'] = None
val_df2['url'] = None
val_df2['labeled_by'] = None

def get_stuff2(row):

    row.id = row.xml.attrib['id']
    row.hyperpartisan = row.xml.attrib['hyperpartisan']
    row.bias = row.xml.attrib['bias']
    row.url = row.xml.attrib['url']
    row.labeled_by = row.xml.attrib['labeled-by']
    return row

val_df2 = val_df2.apply(get_stuff2, axis=1)
val_df2.drop(columns=['xml'], inplace=True)

In [10]:
val_df2.head()

,id,hyperpartisan,bias,url,labeled_by
0,0000001,true,right,https://dailywire.com/news/20858/after-devos-a...,publisher
1,0000002,true,right,https://dailywire.com/news/16103/university-aw...,publisher
2,0000008,false,right-center,https://newsline.com/texas-state-university-su...,publisher
3,0000009,false,least,https://apnews.com/795c45f6db904f8eb85d72cee61...,publisher
4,0000010,true,left,https://dissentmagazine.org/article/eve-and-th...,publisher


In [11]:
val_df = pd.merge(val_df, val_df2, on='id')
val_df.head()

,id,publish_time,title,text,hyperpartisan,bias,url,labeled_by
0,0000001,2017-09-10,After DeVos Announced Plans To Reexamine Title...,When explaining her decision to reevaluate Tit...,true,right,https://dailywire.com/news/20858/after-devos-a...,publisher
1,0000002,2017-05-05,University To Award Trayvon Martin With Posthu...,A Florida university will honor Trayvon Martin...,true,right,https://dailywire.com/news/16103/university-aw...,publisher
2,0000008,2017-11-15,Texas State University suspends Greek life aft...,Nov. 15 (UPI) ? Texas State University has sus...,false,right-center,https://newsline.com/texas-state-university-su...,publisher
3,0000009,2015-06-10,Red Sox waste Rodriguez outing in 1-0 loss to ...,BALTIMORE (AP) ? On a night in which Boston di...,false,least,https://apnews.com/795c45f6db904f8eb85d72cee61...,publisher
4,0000010,,Eve and the New Jerusalem,"Imaginings of a utopian, or utopian socialist,...",true,left,https://dissentmagazine.org/article/eve-and-th...,publisher


In [12]:
val_df.to_csv("/scratch/users/omutlu/hyperpartisan/train.csv",index=False)

In [108]:
asd = pd.read_csv("/scratch/users/omutlu/hyperpartisan/train.csv")
def tonumeric(row):

    if row.hyperpartisan == True:
        row.hyperpartisan = 1
    else:
        row.hyperpartisan = 0

    return row

asd = asd.apply(tonumeric, axis=1)
asd.to_csv("/scratch/users/omutlu/hyperpartisan/train2.csv",index=False)

,id,publish_time,title,text,hyperpartisan,bias,url,labeled_by
0,1,2017-09-10,After DeVos Announced Plans To Reexamine Title...,When explaining her decision to reevaluate Tit...,1,right,https://dailywire.com/news/20858/after-devos-a...,publisher
1,2,2017-05-05,University To Award Trayvon Martin With Posthu...,A Florida university will honor Trayvon Martin...,1,right,https://dailywire.com/news/16103/university-aw...,publisher
2,8,2017-11-15,Texas State University suspends Greek life aft...,Nov. 15 (UPI) ? Texas State University has sus...,0,right-center,https://newsline.com/texas-state-university-su...,publisher
3,9,2015-06-10,Red Sox waste Rodriguez outing in 1-0 loss to ...,BALTIMORE (AP) ? On a night in which Boston di...,0,least,https://apnews.com/795c45f6db904f8eb85d72cee61...,publisher
4,10,NaN,Eve and the New Jerusalem,"Imaginings of a utopian, or utopian socialist,...",1,left,https://dissentmagazine.org/article/eve-and-th...,publisher
5,12,2016-09-26,Jewish Organization's Huge Day Of Unity On Tue...,Against the backdrop of an increasingly polari...,1,right,https://dailywire.com/node/9485,publisher
6,15,2016-11-29,"BREAKING: Trump Reaches Agreement To Keep 1,00...",President-elect Donald Trump has reached an ag...,1,right,https://dailywire.com/news/11184/breaking-trum...,publisher
7,16,2017-12-27,Winning numbers drawn in ?2 By 2? game,"LINCOLN, Neb. (AP) _ The winning numbers in Tu...",0,least,https://apnews.com/46135a1ed45145fb82f654644df...,publisher
8,20,NaN,ANALYSIS: Does gay debate mirror church disput...,WASHINGTON&#160;(RNS) ? One group of Christian...,0,left-center,https://baptistnews.com/article/analysisdoesga...,publisher
9,21,2012-01-06,Slideshow: Salvaging Burned Books in Egypt,"In December, clashes in Egypt between pro-demo...",0,left-center,https://pri.org/stories/2012-01-06/slideshow-s...,publisher


In [112]:
asd = dsa.drop(columns=['id','publish_time','title','bias','url','labeled_by'])
asd.to_csv("/scratch/users/omutlu/hyperpartisan/train3.csv",index=False)

In [98]:
asd = pd.read_csv("/scratch/users/omutlu/hyperpartisan/train3.csv")

In [99]:
asd[asd.hyperpartisan == 0]

,text,hyperpartisan
0,When explaining her decision to reevaluate Tit...,0
1,A Florida university will honor Trayvon Martin...,0
2,Nov. 15 (UPI) ? Texas State University has sus...,0
3,BALTIMORE (AP) ? On a night in which Boston di...,0
4,"Imaginings of a utopian, or utopian socialist,...",0
5,Against the backdrop of an increasingly polari...,0
6,President-elect Donald Trump has reached an ag...,0
7,"LINCOLN, Neb. (AP) _ The winning numbers in Tu...",0
8,WASHINGTON&#160;(RNS) ? One group of Christian...,0
9,"In December, clashes in Egypt between pro-demo...",0


In [1]:
from torchtext.data import Field
import torch

tokenize = lambda x: x.split()
# We can also use nltk.tokenize() or tokenize="spacy" which is default
# Can add  fix_length=N -> Length of longest sequence
#Can add tensor_type=tensor_type (e.g torch.cuda.LongTensor) if we already know we are going to work woth GPU's
#You can add include_lengths=True
Text = Field(sequential=True, tokenize=tokenize, lower=True) # this can be any name e.g "asd"
Label = Field(sequential=False, use_vocab=False, dtype=torch.float)

In [2]:
import csv
import sys

csv.field_size_limit(sys.maxsize)

131072

In [3]:
from torchtext.data import TabularDataset
 
#tv_datafields = [("id", Label), ("publish_time", Label), ("title", Text), ("text", Text),
#                 ("hyperpartisan", Label), ("bias", Label), ("url", Label), ("labeled_by", Label),]
tv_datafields = [("text", Text), ("hyperpartisan", Label)]
trn, vld = TabularDataset.splits(
               path="/scratch/users/omutlu/hyperpartisan", # the root directory where the data lies. train_small.csv and val_small.csv for small amount of data 
               train='train3.csv', validation="val2.csv",
               format='csv',
               skip_header=True, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=tv_datafields)

In [64]:
vld[0].__dict__.keys()

dict_keys(['text', 'hyperpartisan'])

In [84]:
trn[111242].hyperpartisan

'0'

In [55]:
trn[0].text

['when',
 'explaining',
 'her',
 'decision',
 'to',
 'reevaluate',
 'title',
 'ix',
 'guidelines',
 'as',
 'they',
 'pertain',
 'to',
 'sexual',
 'assault',
 'on',
 'college',
 'campuses,',
 'secretary',
 'of',
 'education',
 'betsy',
 'devos',
 'said:',
 '?every',
 'survivor',
 'of',
 'sexual',
 'misconduct',
 'must',
 'be',
 'taken',
 'seriously.',
 'every',
 'student',
 'accused',
 'of',
 'sexual',
 'misconduct',
 'must',
 'know',
 'that',
 'guilt',
 'is',
 'not',
 'predetermined.?',
 'the',
 'obama',
 'administration?s',
 'changes',
 'to',
 'title',
 'ix',
 'have',
 'been',
 'criticized',
 'for,',
 'among',
 'other',
 'things,',
 'substantially',
 'lowering',
 'the',
 'burden',
 'of',
 'proof',
 'as',
 'it',
 'pertains',
 'to',
 'sexual',
 'assault,',
 'as',
 'well',
 'as',
 'denying',
 'elements',
 'of',
 'due',
 'process',
 'to',
 'the',
 'accused.',
 'however,',
 'many',
 'progressives',
 'are',
 'lashing',
 'out',
 'at',
 'devos',
 'because',
 'they',
 'hate',
 'her,',
 'and',


In [4]:
# For a complete list of pretrained vectors -> https://github.com/pytorch/text/blob/v0.2.1/torchtext/vocab.py#L379
#The downloaded word embeddings will stay at ./.vector_cache folder.
# Can also give custom vectors by vectors=vectors
#You can create your own vectors by using this class vocab.Vectors.
Text.build_vocab(trn,max_size=20000,min_freq=5, vectors="fasttext.en.300d")

# Can get
#Text.vocab.freqs
#Text.vocab.itos
#Text.vocab.stoi
#Text.vocab.vectors

In [5]:
Text.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0,
             '<pad>': 1,
             'the': 2,
             'to': 3,
             'of': 4,
             'and': 5,
             'a': 6,
             'in': 7,
             'that': 8,
             'for': 9,
             'is': 10,
             'on': 11,
             'with': 12,
             'as': 13,
             'was': 14,
             'it': 15,
             'by': 16,
             'are': 17,
             'from': 18,
             'be': 19,
             'at': 20,
             'have': 21,
             'he': 22,
             'has': 23,
             'this': 24,
             'but': 25,
             'not': 26,
             'an': 27,
             'his': 28,
             'they': 29,
             'i': 30,
             'or': 31,
             'who': 32,
             'their': 33,
             'we': 34,
             'new': 35,
             'you': 36,
             'will': 37,
             'more': 38,
             'said'

In [103]:
from torchtext.data import Iterator, BucketIterator

#BucketIterator does padding behind the scenes. It does it by the sort_key function that you gave it. In our case number of words in text

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_iter, val_iter = BucketIterator.splits(
 (trn, vld), # we pass in the datasets we want the iterator to draw data from
 batch_sizes=(64, 64),
 device=device, # if you want to use the GPU, specify the GPU number here
 sort_key=lambda x: len(x.text), # the BucketIterator needs to be told what function it should use to group the data.
 sort_within_batch=False,
 repeat=False # we pass repeat=False because we want to wrap this Iterator layer.
)

#repeat=False because we will do it ourselves
#train_iter = Iterator(trn, batch_size=64, device=-1, sort=True, sort_within_batch=False, repeat=False)
#val_iter = Iterator(vld, batch_size=64, device=-1, sort=True, sort_within_batch=False, repeat=False)

In [104]:
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class LSTMClassifier(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, batch_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        #self.device = 0
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # the first is the hidden h
        # the second is the cell  c
        return [autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim).to(device)),
                autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim).to(device))]

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        x = embeds.view(len(sentence), self.batch_size , -1)
        lstm_out, last_hidden = self.lstm(x, self.hidden)
        #lstm_out, last_hidden = self.lstm(x, [i.to(self.device) for i in self.hidden])
        #lstm_out, (hidden, cell) = self.lstm(x)
        
        #lstm_out = [sent len, batch size, hid dim] Concat of all hiddens
        #hidden = [1, batch size, hid dim] Last hidden
        
#        assert torch.equal(lstm_out[-1,:,:], last_hidden.squeeze(0))
        
        return self.hidden2label(lstm_out[-1])
        #return self.hidden2label(hidden.squeeze(0))

#        y  = self.hidden2label(lstm_out[-1])
#        y_prob = y.sigmoid()
#        return y

In [105]:
def get_accuracy(truth, pred):
    assert len(truth)==len(pred)
    correct = 0
    for i in range(len(truth)):
        if truth[i]==pred[i]:
            correct += 1.0
    return correct/len(truth)

def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [106]:
def train_epoch(model, train_iter, loss_function, optimizer, text_field, label_field, i):
    model.train()
    avg_loss = 0.0
    avg_acc = 0.0
    count = 0
    truth_res = []
    pred_res = []
    for batch in train_iter:
        sent, label = batch.text, batch.hyperpartisan
        model.zero_grad() # Pytorch doesn't automatically set the previous batch's grad to zero
#        label.data.sub_(1)
#        truth_res += list(label.data)

#        if torch.cuda.device_count() > 1:
#            model.module.batch_size = len(label.data)
#            model.module.device = device
#            model.module.hidden = model.module.init_hidden()  # detaching it from its history on the last instance.
#        else:
#            model.batch_size = len(label.data)
#            model.hidden = model.init_hidden()# detaching it from its history on the last instance.


        model.batch_size = len(label.data)
        model.hidden = model.init_hidden()  # detaching it from its history on the last instance.
        pred = model(sent).squeeze(1) # (64,1) to 64
#        pred_label = (pred > 0.5)*1
        pred = pred.cpu()
        label = label.cpu()
        #print(label)
        acc = binary_accuracy(pred,label)
        avg_acc += acc.item()
#        pred_res += [x[0] for x in pred_label]
#        label = label.type(torch.FloatTensor)
#        label = label.view(64,1)
        loss = loss_function(pred, label)
        avg_loss += loss.item()
        count += 1
#        if count % 100 == 0:
#            print('epoch: %d iterations: %d loss :%g' % (i, count*model.batch_size, loss.item()))
        loss.backward()
        optimizer.step()

    avg_loss /= len(train_iter)
    avg_acc /= len(train_iter)
    print('epoch: %d done!\ntrain avg_loss:%g , acc:%g'%(i, avg_loss, avg_acc))

In [107]:
def evaluate(model, eval_iter, loss_function):
    model.eval()
    avg_loss = 0.0
    avg_acc = 0.0
    truth_res = []
    pred_res = []
    for batch in eval_iter:
        sent, label = batch.text, batch.hyperpartisan
#        label.data.sub_(1)
#        truth_res += list(label.data)

#        if torch.cuda.device_count() > 1:
#            model.module.batch_size = len(label.data)
#            model.module.device = device
#            model.module.hidden = model.module.init_hidden()  # detaching it from its history on the last instance.
#        else:
#            model.batch_size = len(label.data)
#            model.hidden = model.init_hidden()# detaching it from its history on the last instance.

        model.batch_size = len(label.data)
        model.hidden = model.init_hidden()  # detaching it from its history on the last instance.
        pred = model(sent).squeeze(1) # (64,1) to 64
        pred = pred.cpu()
        label = label.cpu()
        
        acc = binary_accuracy(pred,label)
        avg_acc += acc.item()
#        pred_label = (pred > 0.5)*1
#        pred_res += [x[0] for x in pred_label]
#        label = label.type(torch.FloatTensor)
#        label = label.view(64,1)
        loss = loss_function(pred, label)
        avg_loss += loss.item()

    avg_loss /= len(eval_iter)
    avg_acc /= len(eval_iter)
    print('dev avg_loss:%g train acc:%g' % (avg_loss, avg_acc ))
    return avg_acc

In [108]:
def train(device):
    EMBEDDING_DIM = 300
    HIDDEN_DIM = 50
    EPOCH = 100
    BATCH_SIZE = 2

    best_dev_acc = 0.0

    model = LSTMClassifier(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM,
                           vocab_size=len(Text.vocab),label_size=1,
                            batch_size=BATCH_SIZE)
#    if torch.cuda.device_count() > 1:
#        model = nn.DataParallel(model)
#        model.module.word_embeddings.weight.data = Text.vocab.vectors.to(device)
#    else:
#        model.word_embeddings.weight.data = Text.vocab.vectors.to(device)

    model.word_embeddings.weight.data = Text.vocab.vectors.to(device)
        #model.device = device
#    loss_function = nn.BCELoss()
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr = 1e-3)
    model = model.to(device)
    loss_function = loss_function.to(device)

    no_up = 0
    for i in range(EPOCH):

        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        
        print('epoch: %d start!' % i)
        train_epoch(model, train_iter, loss_function, optimizer, Text, Label, i)
        print('now best dev acc:',best_dev_acc)
        dev_acc = evaluate(model,val_iter,loss_function)
#        if dev_acc > best_dev_acc:
#            best_dev_acc = dev_acc
            #os.system('rm mr_best_model_minibatch_acc_*.model')
#            print('New Best Dev!!!')
            #torch.save(model.state_dict(), 'mr_best_model_minibatch_acc_' + str(int(dev_acc*10000)) + '.model')
#            no_up = 0
#        else:
#            no_up += 1
#            if no_up >= 10:
#                exit()
        
        end.record()

        # Waits for everything to finish running
        torch.cuda.synchronize()

        print("Elapsed time is : %.4f" % (start.elapsed_time(end)/1000))

In [109]:
train(device)

epoch: 0 start!


RuntimeError: CUDA out of memory. Tried to allocate 387.38 MiB (GPU 0; 10.91 GiB total capacity; 10.08 GiB already allocated; 71.31 MiB free; 158.47 MiB cached)

In [2]:
import torch
torch.version.cuda

'9.0.176'

In [14]:
torch.cuda.get_device_name(0)

'GeForce GTX 1080 Ti'